# Learning Algorithm

This project implements a *Value Based* method called [Deep Q-Networks](https://deepmind.com/research/dqn/). 

Deep Q Learning combines 2 approaches :
- A Reinforcement Learning method called [Q Learning](https://en.wikipedia.org/wiki/Q-learning) (aka SARSA max)
- A Deep Neural Network for value function approximation, which is a major technique to deal with continuous state spaces. 

Especially, this implementation includes the 2 major training improvements by [Deepmind](https://deepmind.com) and described in their [Nature publication : "Human-level control through deep reinforcement learning (2015)"](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)
- Experience Replay 
- Fixed Q Targets

> Reinforcement learning is known to be unstable or even to diverge when a nonlinear function approximator such as a neural network is used to represent the action-value (also known as Q) function. This instability has several causes: the correlations present in the sequence of observations, the fact that small updates to Q may significantly change the policy and therefore change the data distribution, and the correlations between the action-values and the target values. We address these instabilities with a novel variant of Q-learning, which uses two key ideas. First, we used a biologically inspired mechanism termed experience replay that randomizes over the data, thereby removing correlations in the observation sequence and smoothing over changes in the data distribution. Second, we used an iterative update that adjusts the action-values towards target values that are only periodically updated, thereby reducing correlations with the target.




# Code implementation

The code used here is adapted from the "LunarLander-v2" tutorial from the Deep Reinforcement Learning Nanodegree, and has been slightly adjusted for being used with the banana environment.

The code consist of :

- model.py : In this python file, a PyTorch QNetwork class is implemented which inherits nn.Module base class. This is a regular fully connected Deep Neural Network using the [PyTorch Framework](https://pytorch.org/docs/0.4.0/). This network will be trained to predict the action to perform depending on the environment observed states. This Neural Network is used by the DQN agent and is composed of :

  - input layer of size 37
  - 2 hidden fully connected layers of 64 cells each
  - output layer which size depends of the action_size parameter passed in the constructor, which is 4 in our problem
  
- dqn_agent.py : In this python file, a DQN agent and a Replay Buffer memory used by the DQN agent) are defined.

  - The DQN agent class is implemented, as described in the Deep Q-Learning algorithm. It provides several methods :
    - constructor : 
      - Initialize the memory buffer (*Replay Buffer*)
      - Initialize 2 instance of the Neural Network : the *target* network and the *local* network
    - step() : 
      - Allows to store a step taken by the agent (state, action, reward, next_state, done) in the Replay Buffer/Memory
      - Every 4 steps (and if their are enough samples available in the Replay Buffer), update the *target* network weights with the current weight values from the *local* network (That's part of the Fixed Q Targets technique)
    - act():
      - It returns actions for the given state as per current policy (Note : The action selection use an Epsilon-greedy selection so that to balance between *exploration* and *exploitation* for the Q Learning)
    - learn():
      - which update the Neural Network value parameters using given batch of experiences from the Replay Buffer. 
    - soft_update():
      - It is called by learn() to slowly blends the weights of the *local* network weights into the *target* Neural Network. 
      - This is an alternative execution of the Fixed Q Targets technique, which is slightly different from the original paper's idea. Instead of wait a few iterations then update the target network, we do a continuous blending here.
    
  - The ReplayBuffer class implements a fixed-size buffer to store experience tuples  (state, action, reward, next_state, done) 
    - add() allows to add an experience step to the memory
    - sample() allows to randomly sample a minibatch of experience steps for the learning  
    
- Navigation.ipynb : This Jupyter notebooks allows to train the agent. More in details it allows to :
  - Import the Necessary Packages 
  - Examine the State and Action Spaces
  - Take Random Actions in the Environment
  - Train an agent using DQN
  - Plot the scores


# DQN parameters and results

The DQN agent uses the following parameters values (defined in dqn_agent.py)

```
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size 
GAMMA = 0.995           # discount factor 
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network
```

The Neural Networks use the following architecture :

```
Input nodes (37) -> Fully Connected Layer (64 nodes, Relu activation) -> Fully Connected Layer (64 nodes, Relu activation) -> Ouput nodes (4)
```

The Neural Networks use the Adam optimizer with a learning rate LR=5e-4 and are trained using a BATCH_SIZE=64.

We are also decreasing the $\epsilon$-greedy parameter in each iteration by a factor of 0.995 and floor it at the level of 0.01 to ensure some minimal exploration in the final stage of learning algorithm. 

After 561 epsiodes of training, the reward per 100 iteration surpassed 13 which marks the succuss of the algorithm. 

# Plot of Rewards
Given the chosen architecture and parameters, our results are :

![Plot of Rewards](./image/BananaTraining.png)

We can see that the learning process is steadily improved as more episodes are passed by, although it is noisy as expected. 

# Ideas for Future Work

Obviously there are still a lot of space to improve efficiency of the algorithm. The Future work can be improved in the following directions:

- HyperParmater tuning: including soft-update blending parameter Tau, Learning Rate, Batch size;
- Neural architecture search: one can test more units/layer, more layers or partially connected layers, etc.
- Implement the state-of-art research algorithms, including the Prioritized Experience Replay and Dueling DQN
